In [1]:
! pip install gdown
! gdown --id 1WO2K-SfU2dntGU4Bb3IYBp9Rh7rtTYEr -O filename

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1WO2K-SfU2dntGU4Bb3IYBp9Rh7rtTYEr
From (redirected): https://drive.google.com/uc?id=1WO2K-SfU2dntGU4Bb3IYBp9Rh7rtTYEr&confirm=t&uuid=d0e28949-aec0-4449-a476-2c5dade25fd5
To: /kaggle/working/filename
100%|█████████████████████████████████████████| 701M/701M [00:02<00:00, 336MB/s]


In [2]:
! pip install h5p

ERROR: Could not find a version that satisfies the requirement h5p (from versions: none)
ERROR: No matching distribution found for h5p


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import h5py
from tqdm import tqdm
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import concurrent.futures

# 1. Custom Loss

In [4]:
class WeightedMSE(nn.Module):
    def __init__(self, weight_nonzero=1000.0):
        super().__init__()
        self.weight_nonzero = weight_nonzero  # Higher weight for non-zero pixels

    def forward(self, y_pred, y_true):
        # Create weight tensor: 1.0 for zeros, `weight_nonzero` for non-zeros
        weights = torch.where(y_true == 0, 
                            torch.tensor(1.0, device=y_true.device),
                            torch.tensor(self.weight_nonzero, device=y_true.device))
        
        # Calculate weighted MSE
        squared_error = (y_true - y_pred) ** 2
        weighted_loss = weights * squared_error
        return torch.mean(weighted_loss)

# 2. Configuration

In [5]:
config = {
    'batch_size': 64,
    'num_epochs': 150,
    'initial_lr': 1e-3,
    'patience': 5,
    'min_lr': 1e-6,
    'num_workers': min(8, os.cpu_count()),  # Limit to 8 workers max
    'pin_memory': True,
    'persistent_workers': True,
    'sparse_threshold': 1e-6,
    'save_full_precision': True,
    'precision_decimals': 8
}

# Set numerical precision
torch.set_printoptions(precision=config['precision_decimals'])
np.set_printoptions(precision=config['precision_decimals'])

# 3. Custom Components

In [6]:
class HandleSparseImages:
    def __call__(self, img):
        sparse_mask = (img < config['sparse_threshold'])
        noise = torch.randn_like(img) * config['sparse_threshold'] * 0.1
        return torch.where(sparse_mask, img + noise, img)

class JetAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 3, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(512*8*8, 4096),
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(4096, 512*8*8),
            nn.ReLU(),
            nn.Unflatten(1, (512, 8, 8)),
            nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.decoder(self.encoder(x))

# 4. Data Loading

In [7]:
def load_data():
    print("Loading data with full precision...")
    with h5py.File('/kaggle/working/filename', 'r') as f:
        X_jets = f['X_jets'][:30000].astype(np.float32)
    
    if config['save_full_precision']:
        np.save('X_jets_original.npy', X_jets)
        print("Saved original data with full precision")
    
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: (x - x.min()) / (x.max() - x.min())),
        HandleSparseImages()
    ])
    
    def process_batch(batch):
        return torch.stack([transform(img) for img in batch])
    
    print("Processing images with all workers...")
    batches = np.array_split(X_jets, config['num_workers'])
    results = []
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=config['num_workers']) as executor:
        futures = [executor.submit(process_batch, batch) for batch in batches]
        for future in tqdm(concurrent.futures.as_completed(futures), 
                         total=len(futures),
                         desc="Processing"):
            results.append(future.result())
    
    return torch.cat(results)

# 5. Training Utilities

In [8]:
def save_reconstructions(loader, epoch, n=5):
    model.eval()
    with torch.no_grad():
        batch = next(iter(loader)).to(device)
        reconstructions = model(batch[:n])
        
        if config['save_full_precision']:
            np.save(f'reconstructions_epoch_{epoch}.npy', reconstructions.cpu().numpy())
        
        fig, axes = plt.subplots(2, n, figsize=(15, 6))
        for i in range(n):
            # Input image
            axes[0,i].imshow(batch[i].cpu().permute(1,2,0))
            axes[0,i].set_title(
                f"Input\nMin: {batch[i].min().item():.{config['precision_decimals']}f}\n"
                f"Max: {batch[i].max().item():.{config['precision_decimals']}f}"
            )
            # Reconstruction
            axes[1,i].imshow(reconstructions[i].cpu().permute(1,2,0))
            axes[1,i].set_title(
                f"Recon\nMin: {reconstructions[i].min().item():.{config['precision_decimals']}f}\n"
                f"Max: {reconstructions[i].max().item():.{config['precision_decimals']}f}"
            )
            axes[0,i].axis('off')
            axes[1,i].axis('off')
        
        plt.tight_layout()
        plt.savefig(f'reconstructions_epoch_{epoch}.png', dpi=300, bbox_inches='tight')
        plt.close()

# 6. Main Training Loop

In [9]:
def main():
    global device, model
    
    # Initialize
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load data
    X_jets = load_data()
    
    # Create datasets
    train_size = int(0.8 * len(X_jets))
    val_size = int(0.1 * len(X_jets))
    test_size = len(X_jets) - train_size - val_size
    
    train_data, val_data, test_data = torch.utils.data.random_split(
        X_jets, [train_size, val_size, test_size])
    
    # Create dataloaders
    train_loader = DataLoader(train_data, batch_size=config['batch_size'], 
                            shuffle=True, num_workers=config['num_workers'],
                            pin_memory=config['pin_memory'],
                            persistent_workers=config['persistent_workers'])
    
    val_loader = DataLoader(val_data, batch_size=config['batch_size'],
                          num_workers=config['num_workers'],
                          pin_memory=config['pin_memory'],
                          persistent_workers=config['persistent_workers'])
    
    test_loader = DataLoader(test_data, batch_size=config['batch_size'],
                           num_workers=config['num_workers'],
                           pin_memory=config['pin_memory'])
    
    # Initialize model
    model = JetAutoencoder().to(device)
    criterion = WeightedMSE()
    optimizer = optim.Adam(model.parameters(), lr=config['initial_lr'])
    scheduler = ReduceLROnPlateau(optimizer, 'min', 
                                patience=config['patience'],
                                min_lr=config['min_lr'],
                                verbose=True)
    writer = SummaryWriter()
    
    # Training
    best_loss = float('inf')
    early_stop_counter = 0
    
    for epoch in range(config['num_epochs']):
        model.train()
        train_loss = 0
        train_pbar = tqdm(train_loader, 
                         desc=f'Epoch {epoch+1}/{config["num_epochs"]} [Train]')
        
        for batch in train_pbar:
            batch = batch.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            outputs = model(batch)
            loss = criterion(outputs, batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_pbar.set_postfix({
                'loss': format(loss.item(), f".{config['precision_decimals']}f"),
                'min': format(batch.min().item(), f".{config['precision_decimals']}f"),
                'max': format(batch.max().item(), f".{config['precision_decimals']}f"),
                'sparse%': f"{(batch < config['sparse_threshold']).float().mean().item() * 100:.2f}"
            })
        
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        val_pbar = tqdm(val_loader,
                       desc=f'Epoch {epoch+1}/{config["num_epochs"]} [Val]')
        
        with torch.no_grad():
            for batch in val_pbar:
                batch = batch.to(device, non_blocking=True)
                outputs = model(batch)
                loss = criterion(outputs, batch)
                val_loss += loss.item()
                val_pbar.set_postfix({
                    'val_loss': format(loss.item(), f".{config['precision_decimals']}f")
                })
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        
        # Logging
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Learning Rate', optimizer.param_groups[0]['lr'], epoch)
        
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Train Loss: {train_loss:.{config['precision_decimals']}f}")
        print(f"Val Loss: {val_loss:.{config['precision_decimals']}f}")
        #print(f"LR: {optimizer.param_groups[0]['lr']:.2e}")
        
        # Save best model
        if val_loss < best_loss:
            best_loss = val_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
            save_reconstructions(test_loader, epoch)
            #print("Saved best model and reconstructions")
        else:
            early_stop_counter += 1
            if early_stop_counter >= config['patience']:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break
    
    # Final save
    torch.save(model.state_dict(), 'final_model.pth')
    writer.close()
    print("Training complete!")

if __name__ == '__main__':
    main()

Using device: cuda
Loading data with full precision...
Saved original data with full precision
Processing images with all workers...


Processing: 100%|██████████| 4/4 [00:33<00:00,  8.47s/it]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/150 [Val]: 100%|██████████| 47/47 [00:01<00:00, 24.31it/s, val_loss=0.50586444]



Epoch 1 Summary:
Train Loss: 12.94556082
Val Loss: 0.50132614


Epoch 2/150 [Val]: 100%|██████████| 47/47 [00:01<00:00, 35.03it/s, val_loss=0.23018356]



Epoch 2 Summary:
Train Loss: 0.30582380
Val Loss: 0.22636931


...


Epoch 84 Summary:
Train Loss: 0.01469774
Val Loss: 0.03233521


Epoch 85/150 [Val]: 100%|██████████| 47/47 [00:01<00:00, 34.82it/s, val_loss=0.03350896]



Epoch 85 Summary:
Train Loss: 0.01459300
Val Loss: 0.03231466


Epoch 86/150 [Val]: 100%|██████████| 47/47 [00:01<00:00, 35.32it/s, val_loss=0.03368169]



Epoch 86 Summary:
Train Loss: 0.01448176
Val Loss: 0.03250176
Early stopping triggered at epoch 86
Training complete!
